In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers
import numpy as np

# Load MNIST dataset
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize images to [-1, 1]

# Hyperparameters
latent_dim = 100
epochs = 10000
batch_size = 128

# Generator model
def build_generator(latent_dim):
    model = models.Sequential([
        layers.Dense(7 * 7 * 128, input_shape=(latent_dim,)),
        layers.Reshape((7, 7, 128)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(1, (7, 7), strides=(1, 1), padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Discriminator model
def build_discriminator():
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=[28, 28, 1]),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002), metrics=['accuracy'])

# Build the generator
generator = build_generator(latent_dim)

# Combined model
gan_input = tf.keras.Input(shape=(latent_dim,))
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)
gan = models.Model(gan_input, gan_output)
discriminator.trainable = False  # Freeze the discriminator
gan.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002))

# Training loop
for epoch in range(epochs):
    noise = tf.random.normal(shape=(batch_size, latent_dim))
    generated_images = generator.predict(noise)
    real_images = train_images[np.random.randint(0, train_images.shape[0], batch_size)]

    # Train discriminator
    discriminator_loss_real = discriminator.train_on_batch(real_images, tf.ones((batch_size, 1)))
    discriminator_loss_fake = discriminator.train_on_batch(generated_images, tf.zeros((batch_size, 1)))
    discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

    # Train generator
    noise = tf.random.normal(shape=(batch_size, latent_dim))
    generator_loss = gan.train_on_batch(noise, tf.ones((batch_size, 1)))

    # Print progress
    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, Discriminator Loss: {discriminator_loss[0]}, Generator Loss: {generator_loss}")


4/4 [==============================] - 1s 225ms/step
Epoch: 0, Discriminator Loss: 3.1252476614899933, Generator Loss: 5.574337959289551
4/4 [==============================] - 0s 90ms/step
Epoch: 100, Discriminator Loss: 0.13307888992130756, Generator Loss: 0.25747478008270264
4/4 [==============================] - 0s 94ms/step
Epoch: 200, Discriminator Loss: 0.01927836611866951, Generator Loss: 6.7679643630981445
4/4 [==============================] - 1s 154ms/step
Epoch: 300, Discriminator Loss: 0.006874244776554406, Generator Loss: 14.738895416259766
4/4 [==============================] - 0s 90ms/step
Epoch: 400, Discriminator Loss: 0.0013835459831170738, Generator Loss: 15.21432113647461
4/4 [==============================] - 0s 90ms/step
Epoch: 500, Discriminator Loss: 0.0012166653905296698, Generator Loss: 15.059537887573242
4/4 [==============================] - 0s 100ms/step
Epoch: 600, Discriminator Loss: 0.0016313343803631142, Generator Loss: 15.246637344360352
4/4 [=========